In [4]:
import json
import random
import pickle

In [5]:
def read_json(file_path):
    data = []
    with open(file_path, "r") as f:
        for json_str in f:  # f is a file object
            data.append(json.loads(json_str))
    return data

In [6]:
json_file_path = "/localscratch/yzhuang43/ra-llm/ToolQA/data/raw_data/dblp/dblp_v14.json"
with open(json_file_path, 'r') as j:
    contents = json.loads(j.read())

In [7]:
print(len(contents))
print(contents[0])

5259858
{'id': '53e99784b7602d9701f3ffdd', 'title': 'Flow.', 'doi': '10.1145/1280120.1280225', 'issue': '10', 'keywords': [], 'lang': 'en', 'venue': {'raw': 'SIGGRAPH Art Gallery'}, 'year': 2006, 'n_citation': 15, 'page_start': '', 'page_end': '', 'volume': '', 'issn': '', 'isbn': '', 'url': [], 'abstract': '', 'authors': [{'id': '53f43776dabfaee0d9b6e75b', 'name': 'Masa Inakage', 'org': ''}], 'doc_type': 'Conference'}


In [8]:
import pandas as pd
data = pd.DataFrame(contents)

In [9]:
data = data.where(data.notnull(), "[BLANK]")

In [10]:
import math
import tqdm
print(len(data))
sub_data = data.loc[(data["references"] != "[BLANK]")]
print(len(sub_data))
sub_data = data.loc[(data["year"] > 2020)]
print(len(sub_data))
sub_data = sub_data.sort_values(by=["n_citation"], ascending=True)
sub_data_paper = sub_data.loc[(sub_data["references"] != "[BLANK]")]
print(len(sub_data))

5259858
3641544
553320
553320


In [11]:
print(contents[0].keys())
print(sub_data.iloc[1])

dict_keys(['id', 'title', 'doi', 'issue', 'keywords', 'lang', 'venue', 'year', 'n_citation', 'page_start', 'page_end', 'volume', 'issn', 'isbn', 'url', 'abstract', 'authors', 'doc_type'])
id                                           623b1afb5aee126c0fdf5169
title                                 Game Awareness: A Questionnaire
doi                                                 10.3390/g12040090
issue                                                               4
keywords            [game awareness,  game elements,  assessment, ...
lang                                                               en
venue                                                {'raw': 'GAMES'}
year                                                             2021
n_citation                                                          0
page_start                                                         90
page_end                                                             
volume                                    

In [12]:
row = sub_data.iloc[-1]
print(row["title"])
print(row["authors"])
print(row["year"])
print(row["venue"])
print(row["n_citation"])
print(row["keywords"])
print(row["doc_type"])
print(row["references"])
print(row["page_start"])
print(row["page_end"])

An Image is Worth 16x16 Words: Transformers for Image Recognition at  Scale
[{'id': '53f430eedabfaee4dc746662', 'name': 'Alexey Dosovitskiy', 'org': 'Google'}, {'id': '53f439efdabfaee2a1d07241', 'name': 'Lucas Beyer', 'org': 'Google'}, {'id': '53f42e76dabfaee4dc72775e', 'name': 'Alexander Kolesnikov', 'org': 'Google'}, {'id': '53f46e13dabfaedf43663cbb', 'name': 'Dirk Weissenborn', 'org': 'Google'}, {'id': '53f43af4dabfaee2a1d12a3f', 'name': 'Xiaohua Zhai', 'org': 'Google'}, {'id': '562c84bd45cedb3398c4b92c', 'name': 'Thomas Unterthiner', 'org': 'Johannes Kepler University of Linz'}, {'id': '562cd89d45cedb3398ce05f8', 'name': 'Mostafa Dehghani', 'org': 'University of Amsterdam'}, {'id': '562ea42245ce1e5967e6d6b2', 'name': 'Matthias Minderer', 'org': 'Google'}, {'id': '53f45a50dabfaefedbb5e7d0', 'name': 'Georg Heigold', 'org': 'German Research Centre for Artificial Intelligence'}, {'id': '5628e21545ce1e59660ab6a8', 'name': 'Sylvain Gelly', 'org': 'Google'}, {'id': '53f44a79dabfaeee22a07a

In [13]:
import networkx as nx

In [14]:
# create mapping between papers and their ids and authors and their ids
id2title_dict = {}
id2author_dict = {}
title2id_dict = {}
author2id_dict = {}
# data_dict = {}
for i in range(len(sub_data)):
    row = sub_data.iloc[i]
    # data_dict[row["id"]] = row
    id2title_dict[row["id"]] = row["title"]
    title2id_dict[row["title"]] = row["id"]
    for j in range(len(row["authors"])):
        author_id = row["authors"][j]["id"]
        name = row["authors"][j]["name"]
        if author_id not in id2author_dict and author_id != '' and author_id != None and author_id != ' ':
            id2author_dict[author_id] = name
            author2id_dict[name] = author_id
print(len(id2title_dict))
print(len(id2author_dict))

553320
355368


In [15]:
author_id1 = random.choice(list(id2author_dict.keys()))
print(author_id1)

63b7dbdddf1d768b3c17882c


In [16]:
data_dict = {}
for i in range(len(data)):
    row = data.iloc[i]
    data_dict[row["id"]] = row

In [17]:
# save dict to file
with open("/localscratch/yzhuang43/ra-llm/ToolQA/data/external_corpus/dblp/title2id_dict.pkl", "wb") as f:
    pickle.dump(title2id_dict, f)
with open("/localscratch/yzhuang43/ra-llm/ToolQA/data/external_corpus/dblp/author2id_dict.pkl", "wb") as f:
    pickle.dump(author2id_dict, f)
with open("/localscratch/yzhuang43/ra-llm/ToolQA/data/external_corpus/dblp/id2title_dict.pkl", "wb") as f:
    pickle.dump(id2title_dict, f)
with open("/localscratch/yzhuang43/ra-llm/ToolQA/data/external_corpus/dblp/id2author_dict.pkl", "wb") as f:
    pickle.dump(id2author_dict, f)

In [18]:
author2id_dict["Chao Zhang"]
print(id2author_dict["632546fd128293c81ede38cc"])
print(id2author_dict["6373427bec88d95668d652c8"])

Chao Zhang
Weihong Lin


In [19]:
# create paper network and author network
paper_net = nx.DiGraph()
author_net = nx.Graph()
print(len(sub_data))
id2node = {}
node2id = {}
nodeid = 0
for j in tqdm.tqdm(range(len(sub_data))):
    context = sub_data.iloc[j]
    id = context["id"]
    paper_net.add_node(id, title=context["title"], authors = context["authors"], year=context["year"], venue=context["venue"], n_citation=context["n_citation"], keywords=context["keywords"], doc_type=context["doc_type"], page_start=context["page_start"], page_end=context["page_end"])
    references = context["references"]
    if references != "[BLANK]":
        for i in range(len(references)):
            if not references[i] in paper_net.nodes():
                if references[i] in data_dict:
                    new_context = data_dict[references[i]]
                    paper_net.add_node(references[i], title=new_context["title"], authors=new_context["authors"], year=new_context["year"], venue=new_context["venue"], n_citation=new_context["n_citation"], keywords=new_context["keywords"], doc_type=new_context["doc_type"], page_start=new_context["page_start"], page_end=new_context["page_end"])
                else:
                    paper_net.add_node(references[i])
            paper_net.add_edge(id, references[i])
            # paper_net.add_edge(references[i], id)
    authors = context["authors"]
    author_list = []
    for i in range(len(authors)):
        if authors[i]["id"] not in author_net.nodes() and authors[i]["id"] != '':
            author_net.add_node(authors[i]["id"], org=authors[i]["org"])
        if authors[i]["id"] != '':
            author_list.append(authors[i]["id"])
    author_list = list(set(author_list))
    for i in range(len(author_list)):
        for k in range(i + 1, len(author_list)):
            if not (author_list[i], author_list[k]) in author_net.edges():
                author_net.add_edge(author_list[i], author_list[k], weight=1, papers=[id], n_citation=[context["n_citation"]])
                author_net.add_edge(author_list[k], author_list[i], weight=1, papers=[id], n_citation=[context["n_citation"]])
            else:
                author_net[author_list[i]][author_list[k]]["weight"] += 1
                author_net[author_list[k]][author_list[i]]["weight"] += 1
                author_net[author_list[i]][author_list[k]]["papers"].append(id)
                author_net[author_list[k]][author_list[i]]["papers"].append(id)
                author_net[author_list[i]][author_list[k]]["n_citation"].append(context["n_citation"])
                author_net[author_list[k]][author_list[i]]["n_citation"].append(context["n_citation"])


    # if j % 10000 == 0:
    #     print(j)


553320


100%|██████████| 553320/553320 [01:36<00:00, 5734.53it/s]


In [20]:
with open('/localscratch/yzhuang43/ra-llm/ToolQA/data/external_corpus/dblp/paper_net.pkl', 'wb') as f:
    pickle.dump(paper_net, f)

with open('/localscratch/yzhuang43/ra-llm/ToolQA/data/external_corpus/dblp/author_net.pkl', 'wb') as f:
    pickle.dump(author_net, f)